pip install git+https://github.com/asimov-academy/Asimov_TV_connector.git

https://python.plainenglish.io/extracting-data-from-trading-view-253919ba7194

In [3]:
from tvDatafeed import TvDatafeed, Interval
from datetime import datetime, timedelta
import pandas as pd

username = open('login_tv.txt', 'r').read()
password = open('passwd_tv.txt', 'r').read()

tv = TvDatafeed(username, password)

ERROR:tvDatafeed.main:error while signin


In [4]:
# tv.search_symbol('PETR4','BMFBOVESPA')
df = tv.get_hist(symbol='PETR4',exchange='BMFBOVESPA',interval=Interval.in_5_minute, n_bars= 5000)
df.head(3)

,symbol,open,high,low,close,volume
datetime,,,,,,
2024-06-20 14:25:00,BMFBOVESPA:PETR4,35.488401,35.527303,35.488401,35.507852,90175.180660
2024-06-20 14:30:00,BMFBOVESPA:PETR4,35.507852,35.527303,35.488401,35.507852,103542.083153
2024-06-20 14:35:00,BMFBOVESPA:PETR4,35.488401,35.507852,35.459225,35.478676,248521.564032


In [5]:
df = tv.get_hist(symbol='PETR4',exchange='BMFBOVESPA',interval=Interval.in_5_minute, n_bars= 10)
df

,symbol,open,high,low,close,volume
datetime,,,,,,
2024-09-06 16:10:00,BMFBOVESPA:PETR4,37.63,37.67,37.60,37.62,653000.0
2024-09-06 16:15:00,BMFBOVESPA:PETR4,37.60,37.63,37.56,37.58,296100.0
2024-09-06 16:20:00,BMFBOVESPA:PETR4,37.58,37.65,37.56,37.63,420100.0
2024-09-06 16:25:00,BMFBOVESPA:PETR4,37.65,37.66,37.62,37.64,369500.0
2024-09-06 16:30:00,BMFBOVESPA:PETR4,37.63,37.65,37.60,37.62,584300.0
2024-09-06 16:35:00,BMFBOVESPA:PETR4,37.62,37.65,37.59,37.61,526400.0
2024-09-06 16:40:00,BMFBOVESPA:PETR4,37.61,37.67,37.60,37.66,461600.0
2024-09-06 16:45:00,BMFBOVESPA:PETR4,37.65,37.68,37.57,37.59,585800.0
2024-09-06 16:50:00,BMFBOVESPA:PETR4,37.58,37.62,37.57,37.57,505600.0


In [7]:
url  = {'ibra50' :'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/Cart_IBr50.csv',
        'ibra100':'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/Cart_IBr100.csv',
        'idiv'   :'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/Cart_Idiv.csv',
        'ibov'   :'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/Cart_Ibov.csv',
        'small'  :'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/Cart_Small.csv'
}

indice = pd.read_csv(url['small'] ,
                     sep=';' , decimal=',' , thousands='.' ,
                     skiprows=1 , skipfooter=2 , encoding='latin-1',
                     index_col=False , engine='python')

indice

,Código,Ação,Tipo,Qtde. Teórica,Part. (%)
0,RRRP3,3R PETROLEUM,ON NM,463981130,3.726
1,TTEN3,3TENTOS,ON NM,112212186,0.442
2,ABCB4,ABC BRASIL,PN N2,73885763,0.579
3,AERI3,AERIS,ON NM,35461088,0.112
4,AESB3,AES BRASIL,ON NM,317101832,1.215
...,...,...,...,...,...
114,VULC3,VULCABRAS,ON NM,96358247,0.594
115,PORT3,WILSON SONS,ON NM,191524900,1.070
116,WIZC3,WIZ CO,ON NM,75354432,0.154
117,YDUQ3,YDUQS PART,ON NM,290010465,0.962


In [8]:
lista = ['PETR3','PETR4','VALE3','BBAS3']

In [9]:
for idx,row in enumerate(lista):
    i=row
    if idx == 0:       
        df =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 100)
        df.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df['symbol']
    else:
        df1 =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 100)
        df1.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df1['symbol']

        df = pd.merge(df,df1,how='inner',left_index=True,right_index=True)

df.to_csv("data/tvohlc.csv" , sep=';' , decimal='.')

In [10]:
def import_data(csv): 
    """ Renomeando as colunas para aritmetica com dataframe """
    def extrair_caracteres(s):
        import re
        pattern = re.compile(r"([^_]+)_")
    
        match = pattern.match(s)
        if match:
                return match.group(1)
        else:
             return None
    
    df = pd.read_csv(csv, sep=';' , decimal='.')
    df.index = pd.to_datetime(df['datetime'])#, format = "%y-%m-%d")
    df = df.drop('datetime', axis=1)
    
    close_columns    = []
    high_columns     = []
    low_columns      = []
    open_columns     = []
    volume_columns   = []

    for i in df.columns:
        if "close" in i:
            close_columns.append(i)
        elif "high" in i:
            high_columns.append(i)
        elif "low" in i:
            low_columns.append(i)
        elif "open_int" in i:
            open_int_columns.append(i)
        elif "open" in i:
            open_columns.append(i)
        elif "volume" in i:
            volume_columns.append(i)

    dicH = {s:extrair_caracteres(s) for s in high_columns}
    dicL = {s:extrair_caracteres(s) for s in low_columns}
    dicO = {s:extrair_caracteres(s) for s in open_columns}
    dicC = {s:extrair_caracteres(s) for s in close_columns}
    dicV = {s:extrair_caracteres(s) for s in volume_columns}
    
    close_df    = df[close_columns].rename(columns=dicC)
    high_df     = df[high_columns].rename(columns=dicH)
    low_df      = df[low_columns].rename(columns=dicL)
    open_df     = df[open_columns].rename(columns=dicO)
    volume_df   = df[volume_columns].rename(columns=dicV)
 
    return [close_df, high_df, low_df, open_df, volume_df]